In [1]:
import os
import pprint as pp

import geopandas as gpd
import matplotlib.pyplot as plt
import nivapy3 as nivapy
import pandas as pd
import teotil3 as teo

In [2]:
# Connect to JupyterHub's PostGIS database
eng = nivapy.da.connect_postgis()

Connection successful.


In [3]:
# Define datasets of interest
# Year for admin. boundaries
admin_year = 2022

reg_gdf = teo.io.get_regine_geodataframe(eng, admin_year)
reg_gdf.head()

,regine,vassom,a_cat_poly_km2,ospar_region,komnr,fylnr,a_agri_km2,a_glacier_km2,a_lake_km2,a_other_km2,a_sea_km2,a_upland_km2,a_urban_km2,a_wood_km2,ar50_tot_a_km2,a_cat_land_km2,a_lake_nve_km2,runoff_mm/yr,q_cat_m3/s,geometry
0,001.10,001,1.442790,Skagerrak,3001,30,0.000000,0.0,0.000013,0.311648,0.281940,0.000000,0.000000,0.849188,0.849201,1.160850,0.000000,592,0.02178,"MULTIPOLYGON (((297006.83 6543966.95, 297169.2..."
1,001.1A1,001,1.432479,Skagerrak,3001,30,0.000000,0.0,0.048098,0.002223,0.000067,0.004615,0.000000,1.377476,1.430189,1.432412,0.043955,620,0.02814,"MULTIPOLYGON (((297505.44 6543157.79, 297543.1..."
2,001.1A20,001,0.340160,Skagerrak,3001,30,0.000000,0.0,0.036623,0.000000,0.000045,0.000000,0.000000,0.303492,0.340114,0.340114,0.000000,594,0.00640,"MULTIPOLYGON (((297770.368 6543429.036, 297787..."
3,001.1A2A,001,17.647822,Skagerrak,3001,30,1.895549,0.0,0.122567,0.000000,0.000000,0.467374,0.131585,15.030746,17.647822,17.647822,0.186340,637,0.35623,"MULTIPOLYGON (((299678.37 6544460.32, 299667.2..."
4,001.1A2B,001,41.298255,Skagerrak,3001,30,1.573851,0.0,7.513687,0.000000,0.000000,2.250799,0.161524,29.798394,41.298255,41.298255,7.344123,637,0.83362,"MULTIPOLYGON (((303353.46 6552989.33, 303341.6..."


In [4]:
# Determine hydrological connectivity
reg_gdf = teo.io.assign_regine_hierarchy(
    reg_gdf,
    regine_col="regine",
    regine_down_col="regine_down",
    order_coastal=False,
    nan_to_vass=True,
    land_to_vass=True,
    add_offshore=True,
)
reg_gdf.head()

100.00 % of regines assigned.


,regine,regine_down,vassom,a_cat_poly_km2,ospar_region,komnr,fylnr,a_agri_km2,a_glacier_km2,a_lake_km2,...,a_sea_km2,a_upland_km2,a_urban_km2,a_wood_km2,ar50_tot_a_km2,a_cat_land_km2,a_lake_nve_km2,runoff_mm/yr,q_cat_m3/s,geometry
0,001.10,001.,001,1.442790,Skagerrak,3001,30,0.000000,0.0,0.000013,...,0.281940,0.000000,0.000000,0.849188,0.849201,1.160850,0.000000,592.0,0.02178,"MULTIPOLYGON (((297006.83 6543966.95, 297169.2..."
1,001.1A1,001.,001,1.432479,Skagerrak,3001,30,0.000000,0.0,0.048098,...,0.000067,0.004615,0.000000,1.377476,1.430189,1.432412,0.043955,620.0,0.02814,"MULTIPOLYGON (((297505.44 6543157.79, 297543.1..."
2,001.1A20,001.1A1,001,0.340160,Skagerrak,3001,30,0.000000,0.0,0.036623,...,0.000045,0.000000,0.000000,0.303492,0.340114,0.340114,0.000000,594.0,0.00640,"MULTIPOLYGON (((297770.368 6543429.036, 297787..."
3,001.1A2A,001.1A20,001,17.647822,Skagerrak,3001,30,1.895549,0.0,0.122567,...,0.000000,0.467374,0.131585,15.030746,17.647822,17.647822,0.186340,637.0,0.35623,"MULTIPOLYGON (((299678.37 6544460.32, 299667.2..."
4,001.1A2B,001.1A2A,001,41.298255,Skagerrak,3001,30,1.573851,0.0,7.513687,...,0.000000,2.250799,0.161524,29.798394,41.298255,41.298255,7.344123,637.0,0.83362,"MULTIPOLYGON (((303353.46 6552989.33, 303341.6..."


In [5]:
# Build network graph from adjacency matrix
g = teo.model.build_graph(reg_gdf, id_col="regine", next_down_col="regine_down")

In [8]:
# Vassdragsområder draining to Trondheimsfjorden
vassoms = range(120, 133)

In [9]:
%%capture

for vassom in vassoms:
    reg_id = f"{vassom:03d}."
    gdf, ax = teo.vis.plot_catchment(
        g, reg_id, reg_gdf, id_col="regine", include_connected=True, direct="up"
    )
    png_path = f"../results/images/vassom_maps/vassom_{vassom:03d}_map.png"
    plt.savefig(png_path, dpi=200, bbox_inches="tight")